### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [4]:
import logging
import os
import sys
import warnings
from urllib.parse import urlparse

import numpy as np
import pandas as pd
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

logging.basicConfig(level = logging.WARN)
logger = logging.getLogger(__name__)

In [5]:
#!pip install mlflow

In [ ]:
!mlflow ui

/home/datascience/.local/lib/python3.8/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2024-05-21 06:42:37 +0000] [21017] [INFO] Starting gunicorn 22.0.0
[2024-05-21 06:42:37 +0000] [21017] [INFO] Listening at: http://127.0.0.1:5000 (21017)
[2024-05-21 06:42:37 +0000] [21017] [INFO] Using worker: sync
[2024-05-21 06:42:37 +0000] [21019] [INFO] Booting worker with pid: 21019
[2024-05-21 06:42:38 +0000] [21020] [INFO] Booting worker with pid: 21020
[2024-05-21 06:42:38 +0000] [21021] [INFO] Booting worker with pid: 21021
[2024-05-21 06:42:38 +0000] [21022] [INFO] Booting worker with pid: 21022


#### Establish Connection to Tracking Server

In [6]:
# Define Username and Password as ENV Variables
os.environ['MLFLOW_TRACKING_USERNAME'] = 'gc35013/oracleidentitycloudservice/mithul.mani@oracle.com'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '}<HNY+Q159O3_(]YW1jr'

In [24]:
# Point to OCI Tracking Server Container
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("demo_model_monitoring")

2024/05/21 06:40:03 INFO mlflow.tracking.fluent: Experiment with name 'demo_model_monitoring' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/473833042123431611', creation_time=1716273603512, experiment_id='473833042123431611', last_update_time=1716273603512, lifecycle_stage='active', name='demo_model_monitoring', tags={}>

In [ ]:
with mlflow.start_run():
    mlflow.log_metric("foo", 1)
    mlflow.log_metric("bar", 2)

In [21]:
!mlflow server --host 0.0.0.0 --port 5000

/home/datascience/.local/lib/python3.8/site-packages/pydantic/_internal/_config.py:334: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)
[2024-05-21 06:29:01 +0000] [19919] [INFO] Starting gunicorn 22.0.0
[2024-05-21 06:29:01 +0000] [19919] [INFO] Listening at: http://0.0.0.0:5000 (19919)
[2024-05-21 06:29:01 +0000] [19919] [INFO] Using worker: sync
[2024-05-21 06:29:01 +0000] [19921] [INFO] Booting worker with pid: 19921
[2024-05-21 06:29:01 +0000] [19922] [INFO] Booting worker with pid: 19922
[2024-05-21 06:29:01 +0000] [19923] [INFO] Booting worker with pid: 19923
[2024-05-21 06:29:01 +0000] [19924] [INFO] Booting worker with pid: 19924
^C
[2024-05-21 06:29:07 +0000] [19919] [INFO] Handling signal: int
[2024-05-21 06:29:07 +0000] [19922] [INFO] Worker exiting (pid: 19922)
[2024-05-21 06:29:07 +0000] [19921] [INFO] Worker exiting (pid: 19921)
[2024-05-21 06:29:07 +0000] [19923] [INFO] Wor

In [22]:
import requests

response = requests.get("http://127.0.0.1:5000")
print(response.status_code)

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb7a2888f70>: Failed to establish a new connection: [Errno 111] Connection refused'))